In [1]:
# Importing Python packages

from astropy.io import fits
import numpy as np
from matplotlib import pylab as plt

# Importing Re-gunning results

In [ ]:
# Setting the path to the folder located in your user where the data is located
redo_path = '/pscratch/sd/p/physedw/redoKP6'

In [ ]:
# Creating a dictionary to store data obtained from deltas
deltas = {
    'ciii': fits.open(f'{redo_path}/deltas/delta-ciii/Log/delta_attributes.fits.gz'),
    'lya': fits.open(f'{redo_path}/deltas/delta-lya/Log/delta_attributes.fits.gz'),
    'lyb': fits.open(f'{redo_path}/deltas/delta-lyb/Log/delta_attributes.fits.gz')
}

# Print basic information to verify that the files are imported
print("=== BASIC INFORMATION ON DELTAS ===")
print(f"{len(deltas)} correlation functions were loaded:")
print()

for name, hdul in deltas.items():
    print(f"📊 {name.upper()}:")
    print("-" * 40)
    hdul.info()
    print()

In [ ]:
# Creating a dictionary to store the obtained correlation functions
correlation_functions = {
    'lyalya': fits.open(f'{redo_path}/correlations/correlation-lyalya/cf_lya_x_lya_exp.fits'),
    'lyalyb': fits.open(f'{redo_path}/correlations/correlation-lyalyb/cf_lya_x_lyb_exp.fits'),
    'qsolya': fits.open(f'{redo_path}/correlations/correlation-qsolya/cf_qso_x_lya_exp.fits'),
    'qsolyb': fits.open(f'{redo_path}/correlations/correlation-qsolyb/cf_qso_x_lyb_exp.fits')
}

# Print basic information to verify that the files are imported
print("=== BASIC INFORMATION ON CORRELATION FUNCTIONS ===")
print(f"{len(correlation_functions)} correlation functions were loaded:")
print()

for name, hdul in correlation_functions.items():
    print(f"📊 {name.upper()}:")
    print("-" * 40)
    hdul.info()
    print()

# Importing Baseline results

In [ ]:
# Setting the path to the folder located of BASELINE KP6 data
baseline_path = '/global/cfs/cdirs/desi/science/lya/y1-kp6/iron-baseline'

In [ ]:
# Creating a dictionary to store data obtained from BASELINE deltas
deltasKP6 = {
    'ciii': fits.open(f'{baseline_path}/deltas/delta-ciii-0-0/Log/delta_attributes.fits.gz'),
    'lya': fits.open(f'{baseline_path}/deltas/delta-lya-0-0/Log/delta_attributes.fits.gz'),
    'lyb': fits.open(f'{baseline_path}/deltas/delta-lyb-0-0/Log/delta_attributes.fits.gz')
}

# Print basic information to verify that the files are imported
print("=== BASIC INFORMATION ON DELTAS ===")
print(f"{len(deltasKP6)} correlation functions were loaded:")
print()

for name, hdul in deltasKP6.items():
    print(f"📊 {name.upper()}:")
    print("-" * 40)
    hdul.info()
    print()

In [ ]:
# Creating a dictionary to store the BASELINE correlation functions
correlation_functions_baseline = {
    'lyalya': fits.open(f'{baseline_path}/correlations/correlation-lyalya-0-0-0/cf_lya_x_lya_exp.fits'),
    'lyalyb': fits.open(f'{baseline_path}/correlations/correlation-lyalyb-0-0-0/cf_lya_x_lyb_exp.fits'),
    'qsolya': fits.open(f'{baseline_path}/correlations/correlation-qsolya-0-0-0/cf_qso_x_lya_exp.fits'),
    'qsolyb': fits.open(f'{baseline_path}/correlations/correlation-qsolyb-0-0-0/cf_qso_x_lyb_exp.fits')
}

# Print basic information to verify that the files are imported
print("=== BASIC INFORMATION ON BASELINE CORRELATION FUNCTIONS ===")
print(f"{len(correlation_functions_baseline)} baseline correlation functions were loaded:")
print()

for name, hdul in correlation_functions_baseline.items():
    print(f"📊 BASELINE {name.upper()}:")
    print("-" * 40)
    hdul.info()
    print()

# Re-running and Baseline deltas

## Exploring Re-running deltas

In [ ]:
# Exploring the contents of the lya .fits
print('--- PRIMARY HEADER ---\n')
for key, value in deltas['lya'][0].header.items():
        print(f"{key:12} : {value}")

for i in range(len(deltas['lya'])):
    hdu = deltas['lya'][i]
    print(f"\n--- {hdu.name} (HDU {i}) ---")
    
    if hasattr(hdu, 'columns') and hdu.columns is not None:
        print(f"Shape: {hdu.data.shape}")
        print("Columns:")
        for col_name in hdu.columns.names:
            col = hdu.columns[col_name]
            print(f"  - {col_name}: format={col.format}, unit={col.unit}")

In [ ]:
# Create subplots for each delta
for name, hdul in deltas.items():
    # Create the same subplot structure for each delta
    f, axs = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
    axs[-1, -1].axis('off')
    
    # Set the main title for the entire figure
    f.suptitle(f'Delta {name.upper()} Analysis', fontsize=16, y=1.02)

    ### Stack Delta Plot
    loglam = hdul['STACK_DELTAS'].data['LOGLAM'][:]
    stack  = hdul['STACK_DELTAS'].data['STACK'][:]
    cut = (stack != 0.) & (hdul['STACK_DELTAS'].data['WEIGHT'][:] > 0.)
    loglam = loglam[cut]
    stack  = stack[cut]
    axs[0][0].plot(10.**loglam, stack, linewidth=1, color='blue')
    axs[0][0].grid(alpha=0.3)
    axs[0][0].set_xlabel(r'$\lambda_{\mathrm{Obs.}} \, [\AA]$', fontsize=12)
    axs[0][0].set_ylabel(r'$\mathrm{\overline{Deltas}}$', fontsize=12)
    axs[0][0].set_title('Stacked Deltas')

    ### ETA Plot
    loglam    = hdul['VAR_FUNC'].data['LOGLAM'][:]
    eta       = hdul['VAR_FUNC'].data['ETA'][:]
    nb_pixels = hdul['VAR_FUNC'].data['NUM_PIXELS'][:]
    cut = (nb_pixels > 0.) & (eta != 1.)
    loglam = loglam[cut]
    eta    = eta[cut]
    axs[0][1].plot(10.**loglam, eta, linewidth=1, color='red')
    axs[0][1].grid(alpha=0.3)
    axs[0][1].set_xlabel(r'$\lambda_{\mathrm{Obs.}} \, [\AA]$', fontsize=12)
    axs[0][1].set_ylabel(r'$\eta$', fontsize=12)
    axs[0][1].set_title('ETA')

    ### VAR_LSS Plot
    loglam    = hdul['VAR_FUNC'].data['LOGLAM'][:]
    var_lss   = hdul['VAR_FUNC'].data['VAR_LSS'][:]
    nb_pixels = hdul['VAR_FUNC'].data['NUM_PIXELS'][:]
    cut       = (nb_pixels > 0.) & (var_lss != 0.1)
    loglam    = loglam[cut]
    var_lss   = var_lss[cut]
    axs[0][2].plot(10.**loglam, var_lss, linewidth=1, color='green')
    axs[0][2].grid(alpha=0.3)
    axs[0][2].set_xlabel(r'$\lambda_{\mathrm{Obs.}} \, [\AA]$', fontsize=12)
    axs[0][2].set_ylabel(r'$\sigma^{2}_{\mathrm{LSS}}$', fontsize=12)
    axs[0][2].set_title('VAR_LSS')

    ### FUDGE Plot
    loglam    = hdul['VAR_FUNC'].data['LOGLAM'][:]
    fudge     = hdul['VAR_FUNC'].data['FUDGE'][:]
    nb_pixels = hdul['VAR_FUNC'].data['NUM_PIXELS'][:]
    cut       = (nb_pixels > 0.) & (fudge != 1.e-7)
    loglam    = loglam[cut]
    fudge     = fudge[cut]
    axs[1][0].plot(10.**loglam, fudge, linewidth=1, color='purple')
    axs[1][0].grid(alpha=0.3)
    axs[1][0].set_xlabel(r'$\lambda_{\mathrm{Obs.}} \, [\AA]$', fontsize=12)
    axs[1][0].set_ylabel(r'$\mathrm{Fudge}$', fontsize=12)
    axs[1][0].set_title('FUDGE')

    ### Mean Continuum Plot
    loglam_rest = hdul['CONT'].data['LOGLAM_REST'][:]
    mean_cont   = hdul['CONT'].data['MEAN_CONT'][:]
    cut = (mean_cont != 0.) & (hdul['CONT'].data['WEIGHT'][:] > 0.)
    loglam_rest = loglam_rest[cut]
    mean_cont   = mean_cont[cut]
    axs[1][1].plot(10.**loglam_rest, mean_cont, linewidth=1, color='orange')
    axs[1][1].grid(alpha=0.3)
    axs[1][1].set_xlabel(r'$\lambda_{\mathrm{R.F.}} \, [\AA]$', fontsize=12)
    axs[1][1].set_ylabel(r'$\mathrm{\overline{Flux}}$', fontsize=12)
    axs[1][1].set_title('Mean Continuum')

    plt.tight_layout()
    plt.show()
    
    # Print some info about each delta
    print(f"=== Delta {name.upper()} Summary ===")
    print(f"Stacked Deltas range: {np.min(stack):.3f} to {np.max(stack):.3f}")
    print(f"ETA range: {np.min(eta):.3f} to {np.max(eta):.3f}")
    print(f"VAR_LSS range: {np.min(var_lss):.3f} to {np.max(var_lss):.3f}")
    print(f"FUDGE range: {np.min(fudge):.3e} to {np.max(fudge):.3e}")
    print(f"Mean Continuum range: {np.min(mean_cont):.3f} to {np.max(mean_cont):.3f}")
    print()

# Close all FITS files when done
for hdul in deltas.values():
    hdul.close()

## Comparison to Baseline Results

In [ ]:
# Plot differences between deltas and deltasKP6 for all three types
for name in deltas.keys():
    if name in deltasKP6:  # Ensure the same delta exists in both dictionaries
        # Create the same subplot structure for differences
        f, axs = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
        axs[-1, -1].axis('off')
        
        # Set the main title for the entire figure
        f.suptitle(f'Differences: Delta {name.upper()} (deltas - deltasKP6)', fontsize=16, y=1.02)

        # Get both HDU lists
        hdul_orig = deltas[name]
        hdul_KP6 = deltasKP6[name]

        ### Stack Delta Difference Plot
        loglam_orig = hdul_orig['STACK_DELTAS'].data['LOGLAM'][:]
        stack_orig = hdul_orig['STACK_DELTAS'].data['STACK'][:]
        cut_orig = (stack_orig != 0.) & (hdul_orig['STACK_DELTAS'].data['WEIGHT'][:] > 0.)
        
        loglam_KP6 = hdul_KP6['STACK_DELTAS'].data['LOGLAM'][:]
        stack_KP6 = hdul_KP6['STACK_DELTAS'].data['STACK'][:]
        cut_KP6 = (stack_KP6 != 0.) & (hdul_KP6['STACK_DELTAS'].data['WEIGHT'][:] > 0.)
        
        # Find common wavelength range
        common_loglam = np.intersect1d(loglam_orig[cut_orig], loglam_KP6[cut_KP6])
        
        if len(common_loglam) > 0:
            # Get values at common wavelengths
            orig_vals = stack_orig[cut_orig][np.isin(loglam_orig[cut_orig], common_loglam)]
            KP6_vals = stack_KP6[cut_KP6][np.isin(loglam_KP6[cut_KP6], common_loglam)]
            difference = orig_vals - KP6_vals
            
            axs[0][0].plot(10.**common_loglam, difference, linewidth=1, color='blue')
            axs[0][0].axhline(y=0, color='red', linestyle='--', alpha=0.7)
            axs[0][0].grid(alpha=0.3)
            axs[0][0].set_xlabel(r'$\lambda_{\mathrm{Obs.}} \, [\AA]$', fontsize=12)
            axs[0][0].set_ylabel(r'$\Delta \mathrm{\overline{Deltas}}$', fontsize=12)
            axs[0][0].set_title('Stacked Deltas Difference')

        ### ETA Difference Plot
        loglam_orig = hdul_orig['VAR_FUNC'].data['LOGLAM'][:]
        eta_orig = hdul_orig['VAR_FUNC'].data['ETA'][:]
        nb_pixels_orig = hdul_orig['VAR_FUNC'].data['NUM_PIXELS'][:]
        cut_orig = (nb_pixels_orig > 0.) & (eta_orig != 1.)
        
        loglam_KP6 = hdul_KP6['VAR_FUNC'].data['LOGLAM'][:]
        eta_KP6 = hdul_KP6['VAR_FUNC'].data['ETA'][:]
        nb_pixels_KP6 = hdul_KP6['VAR_FUNC'].data['NUM_PIXELS'][:]
        cut_KP6 = (nb_pixels_KP6 > 0.) & (eta_KP6 != 1.)
        
        common_loglam = np.intersect1d(loglam_orig[cut_orig], loglam_KP6[cut_KP6])
        
        if len(common_loglam) > 0:
            orig_vals = eta_orig[cut_orig][np.isin(loglam_orig[cut_orig], common_loglam)]
            KP6_vals = eta_KP6[cut_KP6][np.isin(loglam_KP6[cut_KP6], common_loglam)]
            difference = orig_vals - KP6_vals
            
            axs[0][1].plot(10.**common_loglam, difference, linewidth=1, color='red')
            axs[0][1].axhline(y=0, color='red', linestyle='--', alpha=0.7)
            axs[0][1].grid(alpha=0.3)
            axs[0][1].set_xlabel(r'$\lambda_{\mathrm{Obs.}} \, [\AA]$', fontsize=12)
            axs[0][1].set_ylabel(r'$\Delta \eta$', fontsize=12)
            axs[0][1].set_title('ETA Difference')

        ### VAR_LSS Difference Plot
        loglam_orig = hdul_orig['VAR_FUNC'].data['LOGLAM'][:]
        var_lss_orig = hdul_orig['VAR_FUNC'].data['VAR_LSS'][:]
        nb_pixels_orig = hdul_orig['VAR_FUNC'].data['NUM_PIXELS'][:]
        cut_orig = (nb_pixels_orig > 0.) & (var_lss_orig != 0.1)
        
        loglam_KP6 = hdul_KP6['VAR_FUNC'].data['LOGLAM'][:]
        var_lss_KP6 = hdul_KP6['VAR_FUNC'].data['VAR_LSS'][:]
        nb_pixels_KP6 = hdul_KP6['VAR_FUNC'].data['NUM_PIXELS'][:]
        cut_KP6 = (nb_pixels_KP6 > 0.) & (var_lss_KP6 != 0.1)
        
        common_loglam = np.intersect1d(loglam_orig[cut_orig], loglam_KP6[cut_KP6])
        
        if len(common_loglam) > 0:
            orig_vals = var_lss_orig[cut_orig][np.isin(loglam_orig[cut_orig], common_loglam)]
            KP6_vals = var_lss_KP6[cut_KP6][np.isin(loglam_KP6[cut_KP6], common_loglam)]
            difference = orig_vals - KP6_vals
            
            axs[0][2].plot(10.**common_loglam, difference, linewidth=1, color='green')
            axs[0][2].axhline(y=0, color='red', linestyle='--', alpha=0.7)
            axs[0][2].grid(alpha=0.3)
            axs[0][2].set_xlabel(r'$\lambda_{\mathrm{Obs.}} \, [\AA]$', fontsize=12)
            axs[0][2].set_ylabel(r'$\Delta \sigma^{2}_{\mathrm{LSS}}$', fontsize=12)
            axs[0][2].set_title('VAR_LSS Difference')

        ### FUDGE Difference Plot
        loglam_orig = hdul_orig['VAR_FUNC'].data['LOGLAM'][:]
        fudge_orig = hdul_orig['VAR_FUNC'].data['FUDGE'][:]
        nb_pixels_orig = hdul_orig['VAR_FUNC'].data['NUM_PIXELS'][:]
        cut_orig = (nb_pixels_orig > 0.) & (fudge_orig != 1.e-7)
        
        loglam_KP6 = hdul_KP6['VAR_FUNC'].data['LOGLAM'][:]
        fudge_KP6 = hdul_KP6['VAR_FUNC'].data['FUDGE'][:]
        nb_pixels_KP6 = hdul_KP6['VAR_FUNC'].data['NUM_PIXELS'][:]
        cut_KP6 = (nb_pixels_KP6 > 0.) & (fudge_KP6 != 1.e-7)
        
        common_loglam = np.intersect1d(loglam_orig[cut_orig], loglam_KP6[cut_KP6])
        
        if len(common_loglam) > 0:
            orig_vals = fudge_orig[cut_orig][np.isin(loglam_orig[cut_orig], common_loglam)]
            KP6_vals = fudge_KP6[cut_KP6][np.isin(loglam_KP6[cut_KP6], common_loglam)]
            difference = orig_vals - KP6_vals
            
            axs[1][0].plot(10.**common_loglam, difference, linewidth=1, color='purple')
            axs[1][0].axhline(y=0, color='red', linestyle='--', alpha=0.7)
            axs[1][0].grid(alpha=0.3)
            axs[1][0].set_xlabel(r'$\lambda_{\mathrm{Obs.}} \, [\AA]$', fontsize=12)
            axs[1][0].set_ylabel(r'$\Delta \mathrm{Fudge}$', fontsize=12)
            axs[1][0].set_title('FUDGE Difference')

        ### Mean Continuum Difference Plot
        loglam_rest_orig = hdul_orig['CONT'].data['LOGLAM_REST'][:]
        mean_cont_orig = hdul_orig['CONT'].data['MEAN_CONT'][:]
        cut_orig = (mean_cont_orig != 0.) & (hdul_orig['CONT'].data['WEIGHT'][:] > 0.)
        
        loglam_rest_KP6 = hdul_KP6['CONT'].data['LOGLAM_REST'][:]
        mean_cont_KP6 = hdul_KP6['CONT'].data['MEAN_CONT'][:]
        cut_KP6 = (mean_cont_KP6 != 0.) & (hdul_KP6['CONT'].data['WEIGHT'][:] > 0.)
        
        common_loglam_rest = np.intersect1d(loglam_rest_orig[cut_orig], loglam_rest_KP6[cut_KP6])
        
        if len(common_loglam_rest) > 0:
            orig_vals = mean_cont_orig[cut_orig][np.isin(loglam_rest_orig[cut_orig], common_loglam_rest)]
            KP6_vals = mean_cont_KP6[cut_KP6][np.isin(loglam_rest_KP6[cut_KP6], common_loglam_rest)]
            difference = orig_vals - KP6_vals
            
            axs[1][1].plot(10.**common_loglam_rest, difference, linewidth=1, color='orange')
            axs[1][1].axhline(y=0, color='red', linestyle='--', alpha=0.7)
            axs[1][1].grid(alpha=0.3)
            axs[1][1].set_xlabel(r'$\lambda_{\mathrm{R.F.}} \, [\AA]$', fontsize=12)
            axs[1][1].set_ylabel(r'$\Delta \mathrm{\overline{Flux}}$', fontsize=12)
            axs[1][1].set_title('Mean Continuum Difference')

        plt.tight_layout()
        plt.show()
        
        # Print statistics about the differences
        print(f"=== Delta {name.upper()} Difference Statistics ===")
        if len(common_loglam) > 0:
            print(f"Stacked Deltas diff - Mean: {np.mean(difference):.3e}, Std: {np.std(difference):.3e}")
        plt.tight_layout()
        plt.show()

# Re-running and Baseline Correlation Functions

## Exploring Re-running Correlation Functions

In [ ]:
# Generating dictionaries to store the data that we will use to plot the correlations

correlation_data = {}
grid_coordinates = {}

# Processing all correlation functions from dictionary 'correlation_functions'
for name, hdul in correlation_functions.items():
    print(f"Processing {name}...")
    
    # Extracting correlation data, RP coordinate and RT coordinate with respect to line-of-sight (LOS)
    corr_data = hdul['COR'].data['DA']
    rp_data = hdul['COR'].data['RP']
    rt_data = hdul['COR'].data['RT']
    
    # Determine grid size automatically
    # For correlations with QSOs, we use fixed dimensions (100, 50). 
    # For correlations without QSOs, we obtain the dimensions automatically.
    if name.startswith('qso'):
        n_parallel = 100
        n_transverse = 50
        print(f" Using fixed grid dimensions: ({n_parallel}, {n_transverse})")
    else:
        n_parallel = int(np.sqrt(corr_data.size))
        n_transverse = n_parallel
        print(f" Using automatic grid dimensions: ({n_parallel}, {n_transverse})")
    
    # Reshape the correlation data and their coordinates data
    correlation_reshaped = corr_data.reshape(n_parallel, n_transverse)
    rp_reshaped = rp_data.reshape(n_parallel, n_transverse)
    rt_reshaped = rt_data.reshape(n_parallel, n_transverse)
    
    # Storing in empty dictionaries
    correlation_data[name] = {
        'correlation': correlation_reshaped,
        'original_data': corr_data
    }
    
    grid_coordinates[name] = {
        'rp': rp_reshaped,
        'rt': rt_reshaped,
        'original_rp': rp_data,
        'original_rt': rt_data
    }
    
    print(f"  ✓ Processed data: {correlation_reshaped.shape}")
    print(f"  ✓ Correlation rank: {np.min(correlation_reshaped):.3f} a {np.max(correlation_reshaped):.3f}")
    print()

print("✅ All data were processed and stored in dictionaries")

In [ ]:
# Plotting correlation functions in bin space

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Correlation Functions\n', fontsize=16, y=0.95)

correlation_names = ['lyalya', 'lyalyb', 'qsolya', 'qsolyb']

for idx, name in enumerate(correlation_names):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]

    # Getting data from dictionaries
    corr = correlation_data[name]['correlation']
    rp = grid_coordinates[name]['rp']
    rt = grid_coordinates[name]['rt']

    # Calculating the value of the correlation bins to plot
    plot_data = (rt**2 + rp**2) * corr
    
    # Use imshow with extent instead
    extent = [rt.min(), rt.max(), rp.min(), rp.max()]
    mesh = ax.imshow(plot_data, extent=extent, cmap='RdYlBu', 
                    aspect='auto', origin='lower')
    
    cbar = plt.colorbar(mesh, ax=ax)
    cbar.set_label(r'$r^2\xi$($r_{\parallel}$, $r_{\perp}$)', fontsize=12)
    
    ax.set_xlabel('$r_{\perp}$ [Mpc/h]', fontsize=12)
    ax.set_ylabel('$r_{\parallel}$ [Mpc/h]', fontsize=12)
    
    titles = {
        'lyalya': r'$Ly\alpha$ x $Ly\alpha$',
        'lyalyb': r'$Ly\alpha$ x $Ly\beta$', 
        'qsolya': r'$QSO$ x $Ly\alpha$',
        'qsolyb': r'$QSO$ x $Ly\beta$'
    }
    ax.set_title(titles[name], fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=10)

plt.tight_layout()
plt.show()

## Exploring Baseline Correlation Functions

In [ ]:
# Generating dictionaries to store the data that we will use to plot the BASELINE correlations

correlation_data_baseline = {}
grid_coordinates_baseline = {}

# Processing all correlation functions from dictionary 'correlation_functions_baseline'
for name, hdul in correlation_functions_baseline.items():
    print(f"Processing BASELINE {name}...")

    # Extracting correlation data, RP coordinate and RT coordinate with respect to line-of-sight (LOS)
    corr_data = hdul['COR'].data['DA']
    rp_data = hdul['COR'].data['RP']
    rt_data = hdul['COR'].data['RT']
    
    # Determine grid size automatically
    # For correlations with QSOs, we use fixed dimensions (100, 50). 
    # For correlations without QSOs, we obtain the dimensions automatically.
    if name.startswith('qso'):
        n_parallel = 100
        n_transverse = 50
        print(f" Using fixed grid dimensions: ({n_parallel}, {n_transverse})")
    else:
        n_parallel = int(np.sqrt(corr_data.size))
        n_transverse = n_parallel
        print(f" Using automatic grid dimensions: ({n_parallel}, {n_transverse})")
    
    # Reshape the correlation data and their coordinates data
    correlation_reshaped = corr_data.reshape(n_parallel, n_transverse)
    rp_reshaped = rp_data.reshape(n_parallel, n_transverse)
    rt_reshaped = rt_data.reshape(n_parallel, n_transverse)
    
    # Storing in BASELINE empty dictionaries
    correlation_data_baseline[name] = {
        'correlation': correlation_reshaped,
        'original_data': corr_data
    }
    
    grid_coordinates_baseline[name] = {
        'rp': rp_reshaped,
        'rt': rt_reshaped,
        'original_rp': rp_data,
        'original_rt': rt_data
    }
    
    print(f"  ✓ Processed data: {correlation_reshaped.shape}")
    print(f"  ✓ Correlation rank: {np.min(correlation_reshaped):.3f} a {np.max(correlation_reshaped):.3f}")
    print()

print("✅ All BASELINE data were processed and stored in dictionaries")

In [ ]:
# Plotting BASELINE correlation functions in bin space

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Baseline Correlation Functions\n', fontsize=16, y=0.95)

correlation_names = ['lyalya', 'lyalyb', 'qsolya', 'qsolyb']

for idx, name in enumerate(correlation_names):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]
    
    # Getting data from BASELINE dictionaries
    corr = correlation_data_baseline[name]['correlation']
    rp = grid_coordinates_baseline[name]['rp']
    rt = grid_coordinates_baseline[name]['rt']
    
    # Calculating the value of the correlation bins to plot
    plot_data = (rt**2 + rp**2) * corr
    
    # Use imshow with extent instead
    extent = [rt.min(), rt.max(), rp.min(), rp.max()]
    mesh = ax.imshow(plot_data, extent=extent, cmap='RdYlBu', 
                    aspect='auto', origin='lower')
    
    cbar = plt.colorbar(mesh, ax=ax)
    cbar.set_label(r'$r^2\xi$($r_{\parallel}$, $r_{\perp}$)', fontsize=12)
    
    ax.set_xlabel('$r_{\perp}$ [Mpc/h]', fontsize=12)
    ax.set_ylabel('$r_{\parallel}$ [Mpc/h]', fontsize=12)
    
    titles = {
        'lyalya': r'Baseline: $Ly\alpha$ x $Ly\alpha$',
        'lyalyb': r'Baseline: $Ly\alpha$ x $Ly\beta$', 
        'qsolya': r'Baseline: $QSO$ x $Ly\alpha$',
        'qsolyb': r'Baseline: $QSO$ x $Ly\beta$'
    }
    ax.set_title(titles[name], fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=10)

plt.tight_layout()
plt.show()

## Comparisson to Baseline Results

In [ ]:
# Plots of the DIFFERENCES between baseline and redo

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Difference: Redo - Baseline Correlation Functions\n', fontsize=16, y=0.95)

correlation_names = ['lyalya', 'lyalyb', 'qsolya', 'qsolyb']

for idx, name in enumerate(correlation_names):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]
    
    # Getting data from BASELINE dictionaries
    corr_baseline = correlation_data_baseline[name]['correlation']
    rp_baseline = grid_coordinates_baseline[name]['rp']
    rt_baseline = grid_coordinates_baseline[name]['rt']
    
    # Getting data from REDO dictionaries
    corr_redo = correlation_data[name]['correlation']
    rp_redo = grid_coordinates[name]['rp']
    rt_redo = grid_coordinates[name]['rt']
    
    # Calculating the value of the correlation bins to plot

    # difference = (rt_baseline**2 + rp_baseline**2) * (corr_redo - corr_baseline)
    # or
    # ratio = corr_redo/corr_baseline
    
    plot_data_baseline = (rt_baseline**2 + rp_baseline**2) * corr_baseline
    plot_data_redo = (rt_redo**2 + rp_redo**2) * corr_redo
    
    # Calculating the difference: REDO - BASELINE
    difference = plot_data_redo - plot_data_baseline
    
    # Use imshow with extent instead
    extent = [rt_baseline.min(), rt_baseline.max(), rp_baseline.min(), rp_baseline.max()]
    mesh = ax.imshow(difference, extent=extent, cmap='RdBu_r', 
                    aspect='auto', origin='lower')
    
    cbar = plt.colorbar(mesh, ax=ax)
    cbar.set_label(r'$\Delta~ r^2\xi$($r_{\parallel}$, $r_{\perp}$)', fontsize=12)
    
    ax.set_xlabel('$r_{\perp}$ [Mpc/h]', fontsize=12)
    ax.set_ylabel('$r_{\parallel}$ [Mpc/h]', fontsize=12)
    
    titles = {
        'lyalya': r'$\Delta$: $Ly\alpha$ x $Ly\alpha$',
        'lyalyb': r'$\Delta$: $Ly\alpha$ x $Ly\beta$', 
        'qsolya': r'$\Delta$: $QSO$ x $Ly\alpha$',
        'qsolyb': r'$\Delta$: $QSO$ x $Ly\beta$'
    }
    ax.set_title(titles[name], fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=10)

plt.tight_layout()
plt.show()